In [ ]:
import pydicom
import numpy as np
import matplotlib.pyplot as plt

# Path to your DICOM file
dcm_file_path = "/Users/davidkorcak/Documents/Rakathon2025/data/radioprotect/Rackaton Data/SAMPLE_001/RS.1.2.246.352.221.53086809173815688567595866456863246500.dcm"

# Load the DICOM file
try:
    dcm = pydicom.dcmread(dcm_file_path)
except Exception as e:
    print(f"Error loading DICOM file: {e}")
    exit()

In [ ]:
with open("keys", "w") as f:
    s = dcm.keys
    f.write(str(s))

In [ ]:
import os
import pydicom
import sys


def find_rtstruct_files(folder_path):
    """
    Traverses a folder and identifies DICOM files with Modality 'RTSTRUCT'.

    Args:
        folder_path (str): The path to the folder containing DICOM files.

    Returns:
        list: A list of file paths identified as RTSTRUCT files.
    """
    rtstruct_files = []
    print(f"Searching for RT Structure Set (RTSTRUCT) files in: {folder_path}\n" + "=" * 30)

    # Walk through the directory tree
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            # Check if the file might be a DICOM file (basic check)
            if filename.lower().endswith(".dcm"):
                filepath = os.path.join(root, filename)
                try:
                    # Read the DICOM file metadata only (faster)
                    # We don't need pixel data to check the modality
                    ds = pydicom.dcmread(filepath, stop_before_pixels=True)

                    # Check if the Modality tag exists and is 'RTSTRUCT'
                    # Using getattr is safer than direct access in case the tag is missing
                    if getattr(ds, "Modality", None) == "RTSTRUCT":
                        print(f"[*] Found RTSTRUCT file: {filepath}")
                        rtstruct_files.append(filepath)

                except pydicom.errors.InvalidDicomError:
                    # Silently ignore files that are not valid DICOM format
                    # print(f"[!] Skipping non-DICOM or invalid file: {filepath}")
                    pass
                except Exception as e:
                    # Catch other potential errors during file reading
                    print(f"[!] Error reading {filepath}: {e}")

    print("=" * 30 + f"\nSearch complete. Found {len(rtstruct_files)} RTSTRUCT file(s).")
    return rtstruct_files


# --- How to run the script ---
if __name__ == "__main__":
    # *** IMPORTANT: Set the path to your DICOM folder here ***
    # Example: target_directory = "/mnt/data/dicom_dump"
    # Example: target_directory = "C:\\Users\\YourUser\\Desktop\\DICOM_Data"
    target_directory = "./data/radioprotect/Rackaton Data/SAMPLE_001"  # Default: Searches the current directory where the script is
    if not os.path.isdir(target_directory):
        print(f"\nError: The specified folder does not exist: {target_directory}")
    else:
        found_files = find_rtstruct_files(target_directory)
        # You can now use the 'found_files' list if needed
        # For example:
        # if found_files:
        #     first_rtstruct = found_files[0]
        #     print(f"\nProcessing the first found file: {first_rtstruct}")
        #     # Add code here to process the RTSTRUCT file


# Use this to generate contour matplotlibs

```
import os
import numpy as np
import matplotlib.pyplot as plt
import pydicom

# --- Configuration ---
Z_TOLERANCE_MM = 0.5  # Tolerance for matching contour Z-coordinate to slice Z-coordinate
DEFAULT_CONTOUR_COLOR = (1.0, 0.0, 0.0)  # Default color (Red) if ROI color is missing

# --- Create output directory ---
output_dir = "contours_colored_transformed"
os.makedirs(output_dir, exist_ok=True)
print(f"Saving contour plots to: {os.path.abspath(output_dir)}")

# --- Iterate through all contours ---
contours_plotted = 0
rtstruct_dir = os.path.dirname(dcm_file_path)  # Get directory of the RTSTRUCT file

# Check if ROIContourSequence exists
if hasattr(dcm, "ROIContourSequence"):
    # --- Add lookup for ROI Names ---
    roi_names = {}
    if hasattr(dcm, "StructureSetROISequence"):
        for item in dcm.StructureSetROISequence:
            if hasattr(item, "ROINumber") and hasattr(item, "ROIName"):
                roi_names[item.ROINumber] = item.ROIName
    # --- End lookup ---

    for i, roi_contour in enumerate(dcm.ROIContourSequence):
        # --- Get ROI Name using ROINumber ---
        roi_number = roi_contour.ReferencedROINumber
        roi_name = roi_names.get(roi_number, f"Unknown ROI {roi_number}")
        # --- End Get ROI Name ---

        # Get ROI color, normalize to 0-1 range for matplotlib
        roi_color = DEFAULT_CONTOUR_COLOR
        if hasattr(roi_contour, "ROIDisplayColor") and roi_contour.ROIDisplayColor:
            try:
                rgb = roi_contour.ROIDisplayColor
                roi_color = tuple(c / 255.0 for c in rgb)
            except Exception as e:
                print(f"Warning: Could not parse ROIDisplayColor for ROI {i + 1}: {roi_contour.ROIDisplayColor}. Using default. Error: {e}")

        # Check if ContourSequence exists within the ROI
        if hasattr(roi_contour, "ContourSequence"):
            for j, contour_sequence_item in enumerate(roi_contour.ContourSequence):
                # Check if ContourGeometricType is CLOSED_PLANAR
                if hasattr(contour_sequence_item, "ContourGeometricType") and contour_sequence_item.ContourGeometricType == "CLOSED_PLANAR":
                    # Check if ContourData and ContourImageSequence exist
                    if (
                        hasattr(contour_sequence_item, "ContourData")
                        and contour_sequence_item.ContourData
                        and hasattr(contour_sequence_item, "ContourImageSequence")
                        and len(contour_sequence_item.ContourImageSequence) > 0
                        and hasattr(contour_sequence_item.ContourImageSequence[0], "ReferencedSOPInstanceUID")
                    ):
                        contour_data = contour_sequence_item.ContourData
                        ref_sop_uid = contour_sequence_item.ContourImageSequence[0].ReferencedSOPInstanceUID

                        # Reshape contour data into (N, 3) array [x, y, z] in mm (Patient Coordinate System)
                        points_mm = np.array(contour_data).reshape((-1, 3))
                        contour_z_mm = points_mm[0, 2]  # Z-coordinate of the contour plane

                        # --- Construct the path to the referenced CT file ---
                        ct_filename = f"CT.{ref_sop_uid}.dcm"
                        ct_file_path = os.path.join(rtstruct_dir, ct_filename)

                        # Load the referenced CT image
                        try:
                            ct_dcm = pydicom.dcmread(ct_file_path)

                            # --- Check for necessary DICOM tags for coordinate transformation ---
                            if not all(hasattr(ct_dcm, tag) for tag in ["ImagePositionPatient", "PixelSpacing", "pixel_array"]):
                                print(
                                    f"Skipping contour {j + 1} for ROI {i + 1}: Referenced CT {ct_filename} is missing required tags (ImagePositionPatient, PixelSpacing, or pixel_array)."
                                )
                                continue

                            ct_image = ct_dcm.pixel_array
                            ct_origin_mm = np.array(ct_dcm.ImagePositionPatient, dtype=float)  # [x, y, z] of top-left pixel center in mm
                            ct_spacing_mm = np.array(ct_dcm.PixelSpacing, dtype=float)  # [row_spacing, col_spacing] in mm/pixel
                            slice_z_mm = ct_origin_mm[2]

                            # --- Check if contour Z matches slice Z within tolerance ---
                            if abs(contour_z_mm - slice_z_mm) > Z_TOLERANCE_MM:
                                # print(f"Skipping contour {j+1} for ROI {i+1}: Z-coordinate mismatch (Contour: {contour_z_mm:.2f} mm, Slice: {slice_z_mm:.2f} mm)")
                                continue  # Skip this contour, it's not for this slice

                            # --- Print ROI Name ---
                            print(
                                f"\nProcessing ROI {roi_number} ('{roi_name}'), Contour {j + 1} on slice Z={slice_z_mm:.2f} mm (Ref SOP UID: {ref_sop_uid})"
                            )
                            # --- End Print ROI Name ---
                            print(f"Attempting to load CT file: {ct_file_path}")

                            # --- Transform contour points from mm (PCS) to pixel coordinates ---
                            # Pixel coordinate (col) = (PointX_mm - OriginX_mm) / SpacingX_mm
                            # Pixel coordinate (row) = (PointY_mm - OriginY_mm) / SpacingY_mm
                            points_pixels = np.zeros_like(points_mm[:, :2])  # Shape (N, 2)
                            points_pixels[:, 0] = (points_mm[:, 0] - ct_origin_mm[0]) / ct_spacing_mm[1]  # X -> Column index
                            points_pixels[:, 1] = (points_mm[:, 1] - ct_origin_mm[1]) / ct_spacing_mm[0]  # Y -> Row index

                            # --- Plotting ---
                            plt.figure(figsize=(8, 8))
                            plt.imshow(ct_image, cmap="gray", origin="upper")  # origin='upper' is typical for medical images
                            # Plot contour points (col, row). Add the first point to the end to close the loop.
                            plt.plot(
                                np.append(points_pixels[:, 0], points_pixels[0, 0]),  # Columns (X)
                                np.append(points_pixels[:, 1], points_pixels[0, 1]),  # Rows (Y)
                                color=roi_color,  # Use the retrieved ROI color
                                linestyle="-",
                                # --- Update Label ---
                                label=f"ROI {roi_number} ('{roi_name}') Contour {j + 1}",
                                # --- End Update Label ---
                            )
                            plt.title(f"CT Slice with RTSTRUCT Contour\n(SOP UID: {ref_sop_uid}, Z={slice_z_mm:.2f} mm)")
                            plt.xlabel("Pixel Column (X)")
                            plt.ylabel("Pixel Row (Y)")
                            plt.legend()
                            plt.axis("equal")
                            plt.gca().invert_yaxis()  # Often needed if origin='upper'

                            # --- Save the plot ---
                            output_filename = f"contour_roi_{i + 1}_contour_{j + 1}_ct_{ref_sop_uid}.png"
                            output_path = os.path.join(output_dir, output_filename)
                            plt.savefig(output_path)
                            plt.close()  # Close the figure to free memory
                            print(f"Saved plot to: {output_path}")
                            contours_plotted += 1

                        except FileNotFoundError:
                            print(f"Error: Referenced CT file not found at {ct_file_path}")
                        except Exception as e:
                            print(f"Error loading or plotting CT file {ct_filename} or processing contour: {e}")
                            import traceback

                            traceback.print_exc()  # Print detailed traceback for debugging
                        # --- End Plotting/Saving Block ---
else:
    print("RTSTRUCT file does not contain ROIContourSequence.")

print(f"\nFinished processing. Plotted and saved {contours_plotted} contours.")
```

In [ ]:
import SimpleITK as sitk
import os

# Directory containing the DICOM series (CT slices)
# Assuming rtstruct_dir is defined from a previous cell
dicom_directory = "/Users/davidkorcak/Documents/Rakathon2025/data/radioprotect/Rackaton Data/SAMPLE_001"  # Example path
# rtstruct_filepath = dcm_file_path # RTSTRUCT path is already in dcm_file_path

print(f"DICOM Directory for CT Series: {dicom_directory}")
# print(f"RTSTRUCT File (loaded via pydicom): {dcm_file_path}")

# --- Load the CT Image Series ---
ct_image_sitk = None  # Initialize variable
try:
    # Get the list of DICOM files in the directory
    # Filter for files ending with .dcm and attempt to read Modality
    series_reader = sitk.ImageSeriesReader()
    dicom_names = []
    print("Scanning for CT files...")
    for filename in os.listdir(dicom_directory):
        if filename.lower().endswith(".dcm"):
            filepath = os.path.join(dicom_directory, filename)
            try:
                # Read only metadata to check Modality quickly
                file_reader = sitk.ImageFileReader()
                file_reader.SetFileName(filepath)
                file_reader.ReadImageInformation()
                # Check if Modality tag exists before accessing
                if "0008|0060" in file_reader.GetMetaDataKeys():
                    modality = file_reader.GetMetaData("0008|0060").strip()
                    if modality.upper() == "CT":
                        dicom_names.append(filepath)
                        # print(f"  Found CT: {filename}")
            except Exception as e:
                # print(f"  Skipping non-CT or unreadable file: {filename} ({e})")
                pass  # Ignore files that are not CT or cause errors

    if not dicom_names:
        print("Error: No CT DICOM files found in the specified directory.")
    else:
        print(f"Found {len(dicom_names)} potential CT files. Attempting to load series...")
        series_ids = series_reader.GetGDCMSeriesIDs(dicom_directory)
        if not series_ids:
            print(f"Error: Could not find DICOM series IDs in {dicom_directory}")
        else:
            # Assuming the most common series ID is the one we want if multiple exist
            # Or simply taking the first one if only one exists
            # More robust: Filter series_ids based on expected characteristics if known
            target_series_id = series_ids[0]
            print(f"Found Series IDs: {series_ids}. Using: {target_series_id}")
            series_filenames = series_reader.GetGDCMSeriesFileNames(dicom_directory, target_series_id)
            series_reader.SetFileNames(series_filenames)

            # Execute the reader
            ct_image_sitk = series_reader.Execute()
            print("\nSuccessfully loaded CT series using SimpleITK.")
            print(f"  Image Size: {ct_image_sitk.GetSize()}")
            print(f"  Image Spacing: {ct_image_sitk.GetSpacing()}")
            print(f"  Image Origin: {ct_image_sitk.GetOrigin()}")
            print(f"  Image Direction: {ct_image_sitk.GetDirection()}")

except Exception as e:
    print(f"Error loading CT series with SimpleITK: {e}")
    import traceback

    traceback.print_exc()

# --- RTSTRUCT File Handling ---
# RTSTRUCT data (contours, ROI names) is loaded using pydicom in the first cell (variable 'dcm').
# SimpleITK is generally not used for parsing RTSTRUCT contour data directly.
print("\nRTSTRUCT data should be accessed via the 'dcm' variable (loaded with pydicom).")

In [ ]:
import numpy as np
import SimpleITK as sitk

# Ensure ct_image_sitk exists and is a SimpleITK image
if 'ct_image_sitk' in locals() and isinstance(ct_image_sitk, sitk.Image):
    print("Generating point cloud from CT image...")

    # Get image properties
    size = ct_image_sitk.GetSize()      # (x, y, z) order
    origin = ct_image_sitk.GetOrigin()  # (x, y, z) order
    spacing = ct_image_sitk.GetSpacing() # (x, y, z) order
    print(f"  Image Size (x,y,z): {size}")
    print(f"  Image Origin (x,y,z): {origin}")
    print(f"  Image Spacing (x,y,z): {spacing}")

    # Get image data as numpy array (z, y, x) order
    ct_array = sitk.GetArrayFromImage(ct_image_sitk)
    print(f"  Numpy array shape (z,y,x): {ct_array.shape}")

    # Define a threshold (optional: adjust as needed)
    # Example: Use a value slightly above the minimum to exclude pure background
    # Or use a known HU threshold if applicable (e.g., -500 for soft tissue)
    threshold = ct_array.min() + 1 # Simple threshold example
    # threshold = -500 # Example HU threshold for tissue

    point_cloud_physical = []
    point_cloud_indices = []
    point_cloud_intensities = []

    # Iterate through voxels using numpy indices (z, y, x)
    for k in range(ct_array.shape[0]): # z index
        for j in range(ct_array.shape[1]): # y index
            for i in range(ct_array.shape[2]): # x index
                intensity = ct_array[k, j, i]
                # Check if voxel intensity is above threshold
                if intensity > threshold:
                    # Get physical coordinates for this voxel index
                    # TransformIndexToPhysicalPoint expects (x, y, z) index order
                    physical_point = ct_image_sitk.TransformIndexToPhysicalPoint((int(i), int(j), int(k)))
                    point_cloud_physical.append(physical_point)
                    point_cloud_indices.append((i, j, k)) # Store index if needed
                    point_cloud_intensities.append(intensity) # Store intensity if needed

    # Convert the list of points to a NumPy array
    point_cloud_physical_np = np.array(point_cloud_physical)

    print(f"\nGenerated point cloud with {point_cloud_physical_np.shape[0]} points above threshold {threshold}.")

    # Display first few points (optional)
    if point_cloud_physical_np.shape[0] > 0:
        print("First 5 points (x, y, z) in mm:")
        print(point_cloud_physical_np[:5, :])

    # The variable 'point_cloud_physical_np' now holds the (N, 3) array of physical coordinates
    # The spacing is implicitly included in these coordinates.

else:
    print("Error: 'ct_image_sitk' not found or is not a valid SimpleITK image.")
    print("Please ensure the previous cell successfully loaded the CT series.")


In [ ]:
import open3d as o3d
import numpy as np

import matplotlib.pyplot as plt

# Check if the necessary variables exist and have data
if 'point_cloud_physical_np' in locals() and 'point_cloud_intensities' in locals() \
    and isinstance(point_cloud_physical_np, np.ndarray) and point_cloud_physical_np.shape[0] > 0 \
    and isinstance(point_cloud_intensities, list) and len(point_cloud_intensities) == point_cloud_physical_np.shape[0]:

     print(f"Preparing point cloud with {point_cloud_physical_np.shape[0]} points for Open3D visualization...")

     # Convert intensities list to numpy array for efficient processing
     intensities_np = np.array(point_cloud_intensities, dtype=np.float64)

     # Normalize intensities to the range [0, 1]
     min_intensity = intensities_np.min()
     max_intensity = intensities_np.max()
     print(f"Intensity range: [{min_intensity}, {max_intensity}]")

     if max_intensity > min_intensity:
          # Normalize intensities: low intensity -> 0, high intensity -> 1
          normalized_intensities = (intensities_np - min_intensity) / (max_intensity - min_intensity)
     elif intensities_np.shape[0] > 0:
          # Handle case where all intensities are the same
          normalized_intensities = np.ones_like(intensities_np) * 0.5 # Assign a mid-gray color
          print("Warning: All points have the same intensity.")
     else:
          # Handle empty case (already checked, but for safety)
          normalized_intensities = np.array([])
          print("Warning: No intensities to normalize.")

     # Create Open3D PointCloud object
     pcd = o3d.geometry.PointCloud()

     # Assign points
     pcd.points = o3d.utility.Vector3dVector(point_cloud_physical_np)

     # Create colors (grayscale based on normalized intensity)
     # R=G=B = normalized_intensity maps low intensity to black and high intensity to white.
     # Open3D's standard visualizer does not directly support per-point alpha.
     # The grayscale color intensity reflects the user's request for dark/white mapping.
     colors_rgb = np.repeat(normalized_intensities[:, np.newaxis], 3, axis=1)

     # Assign colors
     pcd.colors = o3d.utility.Vector3dVector(colors_rgb)

     # --- Visualization ---
     print("Displaying point cloud. Close the visualization window to continue.")
     # Optional: Add visualization options like point size or background color
     vis = o3d.visualization.Visualizer()
     vis.create_window(window_name="CT Point Cloud - Colored by Intensity")
     vis.add_geometry(pcd)
     opt = vis.get_render_option()
     opt.background_color = np.asarray([0.1, 0.1, 0.1]) # Dark background
     # opt.point_size = 1.0 # Adjust point size if needed
     vis.run()
     vis.destroy_window()

     # --- Optional Downsampling for Large Clouds ---
     # If visualization is slow, uncomment the following lines to downsample:
     # print("Point cloud is large. Consider downsampling for better performance.")
     # voxel_down_size = 1.0 # Voxel size in mm, adjust as needed
     # pcd_downsampled = pcd.voxel_down_sample(voxel_down_size)
     # print(f"Downsampled to {len(pcd_downsampled.points)} points using voxel size {voxel_down_size} mm.")
     # print("Displaying downsampled point cloud...")
     # o3d.visualization.draw_geometries([pcd_downsampled],
     #                                   window_name=f"Downsampled CT Point Cloud (Voxel Size: {voxel_down_size}mm)")

else:
     print("Error: Required variables ('point_cloud_physical_np', 'point_cloud_intensities') not found, are not numpy arrays/lists, or are empty.")
     print("Please ensure the previous cell generating the point cloud ran successfully.")